In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from matplotlib.pyplot import rc_context

import sys
import warnings

import scvi
import anndata
import matplotlib.pyplot as plt
# import scrublet as scr

Global seed set to 0


In [2]:
#set up directory
large_root = r"../data"
results_file = large_root + "/results_file.h5ad"

In [3]:
sc.set_figure_params(dpi=100, color_map = 'viridis_r')
sc.settings.verbosity = 1
sc.logging.print_header()

scanpy==1.9.2 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.10.0 pandas==1.5.3 scikit-learn==1.2.1 statsmodels==0.13.5 python-igraph==0.10.4 pynndescent==0.5.4


In [4]:
index2 = pd.read_csv("../01-data/GSE120575_Sade_Feldman_melanoma_single_cells_TPM_GEO.txt", delimiter="\t",skiprows=[0], nrows=0).T

print(index2.shape)
index2 = index2.iloc[1:,:]
print(index2.shape)
index2 = index2.reset_index()
index2.rename(columns = {'index':'index2'}, inplace = True)

index2.head()
# adata = adata.iloc[1:,:].copy()
# adata.head()

(16292, 0)
(16291, 0)


,index2
0,Pre_P1
1,Pre_P1.1
2,Pre_P1.2
3,Pre_P1.3
4,Pre_P1.4


In [5]:
adata = pd.read_csv("../01-data/GSE120575_Sade_Feldman_melanoma_single_cells_TPM_GEO.txt", delimiter="\t",skiprows=[1]).T

print(adata.shape)
adata = adata.iloc[1:,:]
adata = adata.reset_index()
adata.head()

(16292, 55737)


,index,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,RP11-126K1.9,RP4-671G15.2,OR6R2P,RP11-225H22.7,RP11-731C17.2,RP4-621B10.8,RP11-114I8.4,RP11-180C16.1,AP000230.1,RP11-80H18.4
0,A10_P3_M11,0.0,0.0,9.24,0.0,0.0,0.0,0.0,0.0,1.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A11_P1_M11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A11_P3_M11,0.0,0.0,7.99,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A11_P4_M11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A12_P3_M11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,0.0,0.0


In [7]:
concatenated = pd.concat([index2, adata], axis="columns")

In [6]:
import scipy
from scipy.sparse import csr_matrix

In [8]:
concatenated = scipy.sparse.csr_matrix(concatenated.values)

MemoryError: Unable to allocate 6.77 GiB for an array with shape (55739, 16291) and data type object

In [ ]:
print(concatenated.shape)
concatenated = concatenated.dropna(how='all')
print(concatenated.shape)

concatenated

In [ ]:
# df['C'] = df.assign(index=df.index.astype(str)).agg(''.join, 1)
# concatenated['full_index'] = concatenated.assign(index=concatenated[0].astype(str).agg('_'.join,1)

concatenated['full_index'] = concatenated['index'] + "-" +concatenated['index2']
concatenated = concatenated.drop('index2', axis = 1)
concatenated = concatenated.drop('index', axis = 1)
concatenated = concatenated.set_index("full_index")
concatenated.head()



In [ ]:
concatenated.to_csv('../01-data/concat_data.csv')

In [ ]:
adata = sc.read_csv("../01-data/concat_data.csv")

# print(adata.shape)
# adata = adata.iloc[1:,:]
# adata.head()

In [ ]:
# bdata = sc.read(concatenated)
# bdata = sc.read_text(concatenated)

# bdata.X  = sc.read_text(concatenated)

adata

In [ ]:
adata.var['mt'] = adata.var.index.str.startswith('MT-')
adata

In [ ]:
# adata.raw.var_names = 
gene_list = adata.raw.var_names.tolist()

In [ ]:
adata.var.rename(columns = {'_index':'index','features':'gene_symbols'}, inplace = True)
adata.var = adata.var.set_index("index")
adata.var.head(2)

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes = 3000)

In [ ]:
adata.X

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
pbmc = sc.datasets.pbmc68k_reduced()

In [ ]:
pbmc

In [ ]:
adata_ref = adata.copy()
pbmc_query = pbmc.copy()

In [ ]:
print(len(pbmc.obs.bulk_labels.unique()))
pbmc.obs.bulk_labels.unique()

In [ ]:
sc.pl.umap(pbmc, color="louvain")

In [ ]:
sc.external.pl.sam(adata, projection='X_umap', c=None, cmap='Spectral_r', linewidth=0.0, edgecolor='k', axes=None, colorbar=True, s=10.0)

In [ ]:
sc.external.pl.sam(pbmc, projection='X_umap', c=None, cmap='Spectral_r', linewidth=0.0, edgecolor='k', axes=None, colorbar=True, s=10.0)

In [ ]:
sc.pl.umap(adata, color=['CD8A','IL2RA','IL2RB','IL2RG'],vmax=20)

In [ ]:
#filter to only include HVGs
adata_HVG = adata[:, adata.var.highly_variable]
adata_HVG

In [ ]:
adata_HVG.obsm

In [ ]:
cell_types = adata.obs.cell_type.unique()
cell_types = cell_types.tolist()

In [ ]:
cell_category_dict = {
    'Mast cells': 'B cells',
    'Plasma B cells': 'B cells',
    'B cells': 'B cells',
    'Proliferative B cells': 'B cells',
    'Recently activated CD4 T cells': 'T cells',    
    'Naive-memory CD4 T cells': 'T cells',
    'Effector memory CD8 T cells': 'T cells',
    'Terminally exhausted CD8 T cells': 'T cells',
    'Transitional memory CD4 T cells': 'T cells',
    'T helper cells': 'T cells',
    'Pre-exhausted CD8 T cells': 'T cells',
    'Proliferative T cells': 'T cells',  
    'Cytotoxic CD8 T cells': 'T cells',
    'Naive T cells': 'T cells',
    'Regulatory T cells': 'T cells',
    'Th17 cells': 'T cells',
    'Proliferative monocytes and macrophages':'Monocytes & Macrophages',
    'M2 TAMs': 'TAMs',   
    'SPP1 TAMs': 'TAMs',
    'Proinflamatory TAMs': 'TAMs',
    'NK': 'NK cells',
    'pDC': 'DCs',
    'cDC': 'DCs',
    'mDC': 'DCs',
    'Monocytes': 'Monocytes & Macrophages',      
}
adata.obs['cell_category'] = adata.obs['cell_type'].map(cell_category_dict).astype('category')

In [ ]:
cytokine_genes_dict = {
    'IL2R': ['IL2RA', 'IL2RB', 'IL2RG'],
    'IL10': ['IL10RA', 'IL10RB'],
    'IL12': ['IL12RB2']
}

In [ ]:
tis_genes_dict = {
    'Antigen Presenting': ['PSMB10','CMKLR1'],
    'T cell/ NK cell': ['NKG7', 'CD8A'],
    'IFN Activity': ['CCL5', 'CXCL9', 'CD27', 'CXCR6','IDO1','STAT1'],
    'T Cell Exhaustion': ['TIGIT', 'LAG3', 'CD274','PDCD1LG2','CD276']
}
# excluding '['HLA-DQA1', 'HLA-DRB1', 'HLA-E']'

In [ ]:
gene1 = 'IL2RA'
gene2 = 'CD3E'
gene3 = 'CD8A'

coex = ((adata.raw[:,'{}'.format(gene1)].X.todense() > 2) & (adata.raw[:,'{}'.format(gene2)].X.todense() > 2)) & (adata.raw[:,'{}'.format(gene3)].X.todense() > 2)

coex_list = [item for sublist in coex.tolist() for item in sublist]
adata.obs['CoEx'] = pd.Categorical(coex_list, categories=[True, False])

# adata.obs['CoEx'] = (adata.raw[:,'{}'.format(gene1)].X.todense() > 0) & (adata.raw[:,'{}'.format(gene2)].X.todense() > 0) 
#                     (adata.raw[:,'{}'.format(gene3)].X.todense() > 0)
adata.obs

In [ ]:
# # filter rows for year does not equal to 2002
# >gapminder_not_2002 = gapminder[gapminder['year']!=2002]
# >gapminder_not_2002.shape
# (1562, 6)


coexdata = adata.obs[adata.obs['CoEx']==True]
coexdata
coexdata.groupby("cell_type").count()

In [ ]:
sc.pl.umap(adata, color='CoEx', groups=[True])

In [ ]:
# with rc_context({'figure.figsize': (4.5, 3)}):
#     sc.pl.violin(adata, ['TIGIT', 'LAG3', 'CD274','PDCD1LG2','CD276'])

In [ ]:
sc.pl.matrixplot(adata, tis_genes_dict, 'cell_type', dendrogram=False, cmap='Blues', standard_scale='var', colorbar_title='column scaled\nexpression')


In [ ]:
sc.pl.matrixplot(adata, cytokine_genes_dict, 'cell_type', dendrogram=True, cmap='Blues', standard_scale='var', colorbar_title='column scaled\nexpression')


In [ ]:
# with rc_context({'figure.figsize': (4.5, 3)}):
# ax = sc.pl.stacked_violin(pbmc, marker_genes_dict, groupby = 'clusters')
ax = sc.pl.stacked_violin(adata, marker_genes_dict, groupby='cell_type', swap_axes=False, dendrogram=True)


In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.obs.groupby("patient").count()

In [ ]:
adata

In [ ]:
sc.pl.umap(adata, color = ['cell_type'], frameon = False)

In [ ]:
with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(adata, color='cell_type', legend_loc='right margin',
               legend_fontsize=12, legend_fontoutline=2,frameon=False,
               title='clustering of cells', palette='Set1')